### Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

##### Load data

Download from 
- Hate crimes: https://data.cityofnewyork.us/Public-Safety/NYPD-Hate-Crimes/bqiq-cu78
- DOE high school: https://data.cityofnewyork.us/Education/2019-DOE-High-School-Directory/uq7m-95z8

In [2]:
hate_crime_df = pd.read_csv('hatecrimes.csv')
school_df = pd.read_csv('doe_high_school.csv')
population_df = pd.read_csv('population.csv')

In [3]:
boroughs_mapping = {
    "PATROL BORO MAN SOUTH": "MANHATTAN",
    "PATROL BORO MAN NORTH": "MANHATTAN",
    "PATROL BORO BKLYN SOUTH": "BROOKLYN",
    "PATROL BORO BKLYN NORTH": "BROOKLYN",
    "PATROL BORO QUEENS SOUTH": "QUEENS",
    "PATROL BORO QUEENS NORTH": "QUEENS",
    "PATROL BORO BRONX": "BRONX",
    "PATROL BORO STATEN ISLAND": "STATEN IS",    
}

boroughs_mapping2 = {
    "MANHATTAN": 1,
    "BROOKLYN": 3,
    "QUEENS": 4,
    "BRONX": 2,
    "STATEN IS": 5,
}

##### Hate Crimes

In [4]:
hate_crime_df['borough'] = hate_crime_df['Patrol Borough Name'].apply(lambda x: boroughs_mapping2[boroughs_mapping[x]]).astype(int)
hate_crime_df['full_complaint_id'] = hate_crime_df['Full Complaint ID']
summary_hate_crime_df = hate_crime_df[['borough', 'full_complaint_id']].groupby(['borough'], as_index=False).count().rename(columns={'full_complaint_id': 'hate_count'})
summary_hate_crime_df.to_csv('summary_hate_crime.csv', index=False)

##### DOE High School

In [5]:
zip_code_name = 'ZIP CODE'

In [6]:
school_df['borough'] = school_df['Borough'].apply(lambda x: boroughs_mapping2[x.strip()])
summary_school_df = school_df[['neighborhood', 'zip']].groupby('zip', as_index=False).count().rename(
    columns={
        'neighborhood': 'number_of_schools',
        'zip': zip_code_name
    }
)
summary_school_df.to_csv('summary_school.csv', index=False)

##### Population

In [7]:
population_df.rename(columns={'MODZCTA': zip_code_name}, inplace=True)
summary_pop_df =  population_df[[zip_code_name, 'pop_est']]
summary_pop_df.to_csv('summary_population.csv', index=False)

#### Housing Price

In [8]:
housing_df = pd.read_csv('houseprice.csv')

In [9]:
col_name = ['ZIP CODE', 'BOROUGH', 'SALE PRICE']
housing_df2 = housing_df[col_name].copy()
mark = housing_df2['ZIP CODE'] > 10000
housing_df2 = housing_df2[mark]
housing_df2['log_saleprice'] = np.log(housing_df2['SALE PRICE'])
housing_df2.groupby(['ZIP CODE', 'BOROUGH'], as_index=False).mean()

,ZIP CODE,BOROUGH,SALE PRICE,log_saleprice
0,10001,1,8.861737e+06,14.083968
1,10002,1,3.511681e+06,13.790782
2,10003,1,2.655826e+06,14.053691
3,10004,1,2.222425e+06,14.032427
4,10005,1,1.049661e+07,14.015219
...,...,...,...,...
179,11436,4,3.484724e+05,12.219551
180,11691,4,5.085674e+05,12.442040
181,11692,4,5.074007e+05,12.856616
182,11693,4,3.696392e+05,12.030059


##### Aggregate

In [10]:
all_df = housing_df2.join(summary_pop_df.set_index('ZIP CODE'), on='ZIP CODE') \
    .join(summary_school_df.set_index('ZIP CODE'), on='ZIP CODE') \
    .join(summary_hate_crime_df.set_index('borough'), on='BOROUGH')

In [11]:
all_df['borough_name'] = all_df['BOROUGH'].apply(lambda x: list(boroughs_mapping2.keys())[list(boroughs_mapping2.values()).index(x)])

#### Data Cleaning

In [12]:
all_df['log_pop'] = np.log(all_df['pop_est'])
mark = all_df['number_of_schools'].isnull()
all_df.loc[mark, 'number_of_schools'] = 0

##### Normalize

In [13]:
features = ['log_saleprice', 'log_pop', 'number_of_schools', 'hate_count']
x = all_df[features].values
x_mean = np.nanmean(x, axis=0)
x_std = np.nanstd(x, axis=0)
x = (x - x_mean) / x_std
x

array([[-0.22149592,  0.97985981,  1.69414448, -1.49978415],
       [-0.06990026,  0.97985981,  1.69414448, -1.49978415],
       [ 0.05812157,  0.97985981,  1.69414448, -1.49978415],
       ...,
       [ 0.31157278, -0.66122954, -0.90857579, -1.81400529],
       [ 0.03704054, -0.66122954, -0.90857579, -1.81400529],
       [-5.65385565, -0.66122954, -0.90857579, -1.81400529]])

In [14]:
for feature in features:
    all_df["norm_" + feature] = x[:, features.index(feature)]

In [15]:
all_df.to_csv('all.csv', index=False)